# Regional average

The `average_polygon` process returns the spatial average of a gridded field over one or more regions defined by polygons. 

In this example, we'll fetch the shape of canadian provinces and compute the temperature average over each one. 

In [1]:
import os
import tempfile
import datetime as dt
from birdy import WPSClient
from owslib.wfs import WebFeatureService
import geopandas as gpd

# Connection to Finch server (local or remote)
url = os.environ.get("WPS_URL", "https://pavics.ouranos.ca/twitcher/ows/proxy/finch/wps")
finch = WPSClient(url)

# Connection to GeoServer storing administrative boundaries
geo = WebFeatureService("http://pavics.ouranos.ca/geoserver/wfs", version="2.0.0")

Get the contour of three canadian provinces.

In [2]:
layer_id = 'public:canada_admin_boundaries'
print(geo.contents[layer_id].title)
resp = geo.getfeature(typename=layer_id, bbox=(-67, 43, -60, 46), outputFormat="application/json")
features = gpd.read_file(resp)

Canada Administrative Boundaries


Note that Finch limits the size of input files. Here, we're simplifyng the geometry to reduce the size of the shape file. In the future, we'll add support to simply send the WFS URL request. 

In [3]:
# Write simplified geometry to file
simplified = features.set_geometry(features.simplify(0.1))
display(simplified)

f = tempfile.NamedTemporaryFile(mode="w", suffix=".geojson")
size = f.write(simplified.to_json())
f.seek(0)
print(f"{f.name}\t {size/1024:.1f}Kb")

,id,PRUID,PRNAME,PRENAME,PRFNAME,PREABBR,PRFABBR,geometry
0,canada_admin_boundaries.4,11,Prince Edward Island / le-du-Prince-douard,Prince Edward Island,le-du-Prince-douard,P.E.I.,.-P.-.,"MULTIPOLYGON (((-62.49300 46.02520, -62.49340 ..."
1,canada_admin_boundaries.9,13,New Brunswick / Nouveau-Brunswick,New Brunswick,Nouveau-Brunswick,N.B.,N.-B.,"MULTIPOLYGON (((-66.76540 44.58380, -66.76870 ..."
2,canada_admin_boundaries.13,12,Nova Scotia / Nouvelle-cosse,Nova Scotia,Nouvelle-cosse,N.S.,N.-.,"MULTIPOLYGON (((-65.64770 43.40800, -65.64930 ..."


/tmp/tmpyate13_u.geojson	 94.1Kb


In [4]:
era5_url = "https://pavics.ouranos.ca/twitcher/ows/proxy/thredds/dodsC/datasets/reanalyses/era5.ncml"
resp = finch.average_polygon(resource=era5_url,
                      shape=f.name, 
                      tolerance=0.1, 
                      start_date=dt.datetime(2000, 1, 1), 
                      end_date=dt.datetime(2000, 1, 10), 
                      variable="tas")

 owslib.wps.WPSException : {'code': 'NoApplicableCode', 'locator': 'None', 'text': 'Process failed, please check server error log'}


In [5]:
out = resp.get(asobj=True).output
out

ProcessFailed: Sorry, process failed.